# *Script* de Treino

## Instalação de dependências

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
%%capture
!pip install OmegaConf
!pip install diffusers
!pip install ftfy
!pip install accelerate==0.16.0
!pip install tensorflow-gpu
!pip install pytorch_lightning
!pip install bitsandbytes
!pip install nvidia-pyindex
!pip install --upgrade nvidia-tensorrt
!pip install --upgrade transformers

## Clone do repositório 
#### Repositório que contém as *scripts* do Dreambooth

In [ ]:
!git clone https://github.com/CrazyBoyM/dreambooth-for-diffusion.git

#### Mudar *path* para a pasta clonada

In [ ]:
%cd /kaggle/working/dreambooth-for-diffusion

## Modelo
#### Download do checkpoint do modelo Stable diffusion 1.4 original da CompVis

In [ ]:
!wget https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4-full-ema.ckpt -P ./ckpt_models

#### Tornar o checkpoint do modelo compativel com a livraria de difusores do Huggingface

In [ ]:
!python tools/ckpt2diffusers.py \
    --checkpoint_path=./ckpt_models/sd-v1-4-full-ema.ckpt \
    --dump_path=./model \
    --original_config_file=./ckpt_models/model.yaml \
    --scheduler_type="ddim"

### Criar pasta para guardar dataset

In [ ]:
!mkdir datasets/data

## Dataset
#### Fazer download do dataset modern-logo-dataset de logo-wizard e guardar cada imagem sendo o nome do ficheiro a sua descrição

In [ ]:
from datasets import load_dataset
from PIL import Image
import os
import io

dataset = load_dataset("logo-wizard/modern-logo-dataset")
output_folder = 'datasets/data'

for i, example in enumerate(dataset["train"]):
    image = example["image"]
    image.save(os.path.join(output_folder, example["text"][:min(len(example["text"]), 200)].replace("/","-")+".png"))

## Fine-tuning
#### Fazer o fine-tuning do modelo em meia precisão com resolução 512x512 no dataset escolhido com a class "logo", learning rate de 1e-6, batch size de 1 e 1000 train steps

In [ ]:
%%time
!accelerate launch tools/train_dreambooth.py \
  --pretrained_model_name_or_path="./model"  \
  --mixed_precision="fp16" \
  --instance_data_dir="/kaggle/working/dreambooth-for-diffusion/datasets/data" \
  --class_data_dir="/kaggle/working/dreambooth-for-diffusion/datasets/data" \
  --class_prompt="logo" \
  --use_8bit_adam \
  --use_filename_as_label \
  --output_dir="./new_model" \
  --center_crop \
  --resolution=512 \
  --train_batch_size=1 \
  --use_8bit_adam \
  --gradient_accumulation_steps=1 --gradient_checkpointing \
  --learning_rate=1e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=1000 \
  --save_model_every_n_steps=1000

## Teste 
#### Testar o modelo ao gerar 3 imagens para cada prompt como forma de testar o modelo

In [ ]:
from diffusers import StableDiffusionPipeline
import torch
import matplotlib.pyplot as plt
from diffusers import DDIMScheduler

model_path = "./new_model"
prompt = "a logo of a babershop, a man beard, black foreground, white background, minimalism, modern"
prompt = "a logo of a coffeeshop, a red mug, red foreground, white background, minimalism, modern"
prompt = "a logo of a parrot, minimalism, modern"
prompt = "a logo of a cow, minimalism, modern"
prompt = "a logo of a car company, front view of a blue car, minimalism, modern"

pipe = StableDiffusionPipeline.from_pretrained(
        model_path, 
        torch_dtype=torch.float16,
        scheduler=DDIMScheduler(
            beta_start=0.00085,
            beta_end=0.012,
            beta_schedule="scaled_linear",
            clip_sample=False,
            set_alpha_to_one=True,
        ),
        safety_checker=None
    )

pipe = pipe.to("cuda")
images = pipe(prompt, num_inference_steps=50, guidance_scale=15.0, num_images_per_prompt=3).images
for i, image in enumerate(images):
    plt.subplot(1,3,i+1)
    plt.imshow(image)
    image.save(f"test-{i}.png")

#### Zipar o modelo para fazer download

In [ ]:
!zip -r logo_model new_model